## 로지스틱 회귀분석

1. Logistic Regression 기본 배경

    - 참/거짓으로 분류

    - 이분으로 분류 : 합격인가 불합격인가
    
    - 로지스틱 회귀 분석은 연속적인 값을 가지더라고, 로지스틱 함수의 결과값은 0과 1 사이의 값을 갖기에 이분법적인 분류 문제 해결하는 모형으로 적합하다
    
    - 이름은 회귀이지만 분류 모형이다
    

2. LOGIT 적용분야
    - 기업 부도 예측
    - 주가-환율-금리 예측
    
[참고] [로지스틱 회귀](https://thebook.io/080228/part02/ch05/01/)
<img src='./imgs/logistic1.png' width='500'>
<img src='./imgs/logistic2.png' width='500'>


[참고] [시그모이드 함수](https://thebook.io/080228/part01/ch02/06/)

[참고] [로지스틱회귀에서 퍼셉트론으로](https://thebook.io/080228/part02/ch05/)
<img src='./imgs/logistic3.png' width='500'>
<img src='./imgs/logistic4.png' width='400'>

[그림] 모두의 딥러딩

이 퍼셉트론은 그 후 인경신경망, 오차역전파 등의 발전을 거쳐 딥러닝으로 이어지게 된다.

#### 타이타닉 데이타

성별(Sex), 나이(Age), 객실등급(Pclass), 요금(Fare)가 생존에 어느 정도의 영향을 미치는가

In [3]:
import pandas as pd
import numpy as np

# 데이타 로딩
df = pd.read_csv('../data/titanic/temp/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


데이타처리

    1) 나이, 요금은 숫자형으로 그래도 사용

    2) 성별과 male/ female로 바로 처리할 수 없다 

    3) 객실등급은 1, 2, 3 으로 이루어져 있지만, 3등석이 높은 값으로 처리하여 결과가 외곡될 수 있다.

[참고] dummy column --> One hot encoding 방식

In [5]:

cols_to_keep = ['Survived','Age','Fare']

dummy_Sex = pd.get_dummies( df['Sex'], prefix='Sex')
print(dummy_Sex.head())

dummy_Pclass = pd.get_dummies( df['Pclass'], prefix='Pclass')
print(dummy_Pclass.head())




   Sex_female  Sex_male
0       False      True
1        True     False
2        True     False
3        True     False
4       False      True
   Pclass_1  Pclass_2  Pclass_3
0     False     False      True
1      True     False     False
2     False     False      True
3      True     False     False
4     False     False      True


In [6]:

data = df[ cols_to_keep]
data.head()

data = data.join( dummy_Pclass)
data.head()


data = data.join( dummy_Sex)
data.head()



,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
0,0,22.0,7.2500,False,False,True,False,True
1,1,38.0,71.2833,True,False,False,True,False
2,1,26.0,7.9250,False,False,True,True,False
3,1,35.0,53.1000,True,False,False,True,False
4,0,35.0,8.0500,False,False,True,False,True


In [7]:
train_label = data['Survived']
train_data = data[ data.columns[1:] ]

print(train_label)
print(train_data)





0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64
      Age     Fare  Pclass_1  Pclass_2  Pclass_3  Sex_female  Sex_male
0    22.0   7.2500     False     False      True       False      True
1    38.0  71.2833      True     False     False        True     False
2    26.0   7.9250     False     False      True        True     False
3    35.0  53.1000      True     False     False        True     False
4    35.0   8.0500     False     False      True       False      True
..    ...      ...       ...       ...       ...         ...       ...
886  27.0  13.0000     False      True     False       False      True
887  19.0  30.0000      True     False     False        True     False
888  30.0  23.4500     False     False      True        True     False
889  26.0  30.0000      True     False     False       False      True
890  32.0   7.7500     False     False      True       False      True

[891 r

In [8]:
# 모델을 선택해서 훈련 데이터를 학습
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_data, train_label)   #학습하는 함수는 fit 이다 (많은 모델 학습)

LogisticRegression()

In [9]:
# 검증하기

print('훈련데이터 정확도: {:.2f}'.format( lr.score(train_data, train_label)))

훈련데이터 정확도: 0.80


In [38]:
print('''
Age  	    Fare	      Pclass_1	            Pclass_2	
Pclass_3	Sex_female	  Sex_male''')
print(lr.coef_)


Age  	    Fare	      Pclass_1	            Pclass_2	
Pclass_3	Sex_female	  Sex_male
[[-3.19993678e-02  1.08759844e-03  1.06275684e+00  5.09495016e-02
  -1.11265010e+00  1.27313241e+00 -1.27207616e+00]]


#### 분석결과

* coef(회귀계수)를 기준으로 생존 확인

* coef 수가 클수록 비례하고 음의 값이면 반비례하는 것이다.


    - 나이가 적을 수록 생존확률이 높다
    - 요금이 높을 수록 생존확률이 높다
    - 객실등급에 따라 1등급 > 2등급 > 3긍급 순으로 생존확률이 높다
    - 여자가 남자보다 생존 확룔이 높다 ( Sex_male은 남성값이기에 음수는 여성이 생존확률이 높다는 것이다 )

* 객실등급과 성별이 생존에 많은 영향을 미쳤다


In [11]:
# 기존 전체 데이타로 예측하기

data['predict'] = lr.predict(train_data)
data.head(20)



,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,predict
0,0,22.0,7.2500,False,False,True,False,True,0
1,1,38.0,71.2833,True,False,False,True,False,1
2,1,26.0,7.9250,False,False,True,True,False,1
3,1,35.0,53.1000,True,False,False,True,False,1
4,0,35.0,8.0500,False,False,True,False,True,0
5,0,30.0,8.4583,False,False,True,False,True,0
6,0,54.0,51.8625,True,False,False,False,True,0
7,0,2.0,21.0750,False,False,True,False,True,0
8,1,27.0,11.1333,False,False,True,True,False,1
9,1,14.0,30.0708,False,True,False,True,False,1


In [12]:
import numpy as np

sample_x = np.array([[28.0, 16.0000,	False, True, False,	False,	True]])

sample_y = lr.predict( sample_x)
sample_y 

C:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([0], dtype=int64)

Survived 와 prdeict를 비교한다.

- Survived=1이면 생존이고 predict 0.5 이상의 값이면 생존으로 예측한 것이다.

- Survived=0이면 사망이고 predict 0.5 미만의 값이면 사망으로 예측한 것이다.

- 결과 확인 중 잘못 예측한 것도 있다 ( 14번째 )